<a href="https://colab.research.google.com/github/luislauriano/Data_Science/blob/master/Projeto_construcao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dicas do Meigarom (YTB)** 

Luis, vamos lá então.

* O projeto que eu mais gostei foi o de Detecção de Cancer de Mama.
* Para resolve qualquer projeto de ML você precisa de 10 passos e você fez 5 deles. Conferir os passos de um projeto de ML novamente
* Gostei das suas explicações depois de cada código.
* Você poderia ter feito uma análise estatística descritiva dos dados, para dar um visão geral dos dados, em termos de métricas estatísticas ( média, mediana, std, moda, … )
* Você poderia ter feito mais análise exploratória de dados. Quais as features mais importantes para o modelo? Qual o impacto de cada feature na variável resposta?
* Você poderia explicar porque usou StandardScaler nas Features. Porque padronizar os dados? O que isso ajuda no modelo?
* Porque você usou LabelEncoder na variável resposta? O que isso traz de benefício?
* Poderia explicar, porque voce usou Random Forest. Qual o motivo da sua escolha?
* Explicar mais sobre as métricas, o que significa cada métrica? Qual das métricas eu deveria olhar? Todas? Apenas Uma?
13h52
* Vai se aprofundando nos projetos e procurando responder porque você está colocando cada coisa. Porque Random forest? Porque Analise Exploratoria? Porque StandarScaler? Para que serve a matrix de correlação?
* Vai fazendo testes e tentando entender o real motivo de cada linha código que você coloca, e porque essa linha é importante e no que ela ajuda?

# **Projetos como base:**

### Projeto do Rafael
https://github.com/barbosarafael/Projetos/blob/master/iNeuron_Back_order_prediction_Notebook/hackaton_ineuron_back_order_prediction.ipynb

### Projeto do Carlos Melo
https://github.com/carlosfab/data_science/blob/master/Setembro_Amarelo.ipynb

### Projeto do Cancer de Mama 
https://github.com/luislauriano/Data_Science/blob/master/Machine_Learning_para_Detec%C3%A7%C3%A3o_do_C%C3%A2ncer_de_Mama.ipynb

#**Referencias:**

Limpeza e preparação dos dados: https://medium.com/@lucasoliveiras/limpeza-e-prepara%C3%A7%C3%A3o-dos-dados-com-pandas-856e844abfbb

Tutorial API do Spotify: https://dandaramcsousa.github.io/2020/05/20/tutorial-api-spotify.html

Escrever e detalhar o maximo e tudo possivel com calma. (Isso também vai ser importante para aprender)   

Explicar cada coisa usada como quando eu for usar o sklearn (para isso vou precisar pesquisar sobre metric score, acuracia e etc para falar sobre)

___________________________________________________________________________________________________


# **Coletando e analisando os dados dos albumns do Spotify de NexoAnexo** 

ou 

# **Spotify & Python & Data Science (Da coleta de dados do Spotify ao Deploy)**

## **Objetivo do projeto (Solução para o problema)**

contexto

A ideia do projeto é fazer uma analise de dados dos albumns que estão no spotify do artista NexoAnexo

## **Etapas do projeto**

* Coleta do dados 
   1. Api do spotify
   2. Dicionario dos dados
   3. Importar os pacotes e dataset necessários
   4. Leitura do arquivo

* Limpeza e preparação dos dados
    1. Deletar colunas
    2. Renomear colunas 
    3. Verificar os tipos das colunas
    4. Alterar os tipos de colunas 
    5. Verificar dados ausentes
    6. Preencher dados ausentes
    7. Corrigindo dados divergentes
    8. Verificando valores duplicados
    9. Outliers
  
* Exploração e visualização dos dados 
* Deploy com python (Streamlit + Heroku)

## **Coleta dos dados**



contexto

Falar um pouco sobre web scraping...

Para coletar os dados vamos utilizar Python e sua biblioteca Spotipy....

### **Explicar toda etapa da coleta dos dados da API do Spotify**

1. Coletar as informações de um artista, dado um nome;
2. Coletar os ids e nomes de todos os álbuns de um artista;
3. Coletar informações de todas as músicas de um álbum;
4. Coletar informações de todas as músicas de todos os álbuns de um artista;
5. Converter tudo para um csv.


In [180]:
!pip install spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import sys
import csv
 
sp = spotipy.Spotify(client_credentials_manager = SpotifyClientCredentials(client_id="dcffc14978ed497aa58834a7864a99b7", client_secret="e8c5cc78c8e74ccd92323d146c635b47"))

In [181]:
def get_artist(name):
    results = sp.search(name)
    items = results['tracks']['items']
    if len(items) > 0:
        return items[0]['artists'][0]
    else:
        return None

In [182]:
def get_artist_id(artist):
    return artist['id']

In [183]:
def get_artist_albums_id_names(id):
  albums = sp.artist_albums(id, country = 'BR', limit=3)
  albums_id_name = {}
  for i in range(len(albums['items'])):
    id = albums['items'][i]['id']
    name = albums['items'][i]['name']
    albums_id_name[id] = name
 
  return albums_id_name

Determinei o limit 3, tendo em vista que Nexo tem 3 albumns, se quisessemos visualizar as outras músicas poderiamos aumentar esse limit para 50, total de musicas que ele possui no spotify.

In [184]:
def get_album_songs(album_id, album_name):
  spotify_album = {}
 
  tracks = sp.album_tracks(album_id)
  
  for n in range(len(tracks['items'])):
    id_track = tracks['items'][n]['id']
    track = sp.track(id_track)
    spotify_album[id_track] = {}
    
    spotify_album[id_track]['album'] = album_name
    spotify_album[id_track]['album_type'] = track['album']['album_type']
    spotify_album[id_track]['track_number'] = track['track_number']
    spotify_album[id_track]['id_track'] = track['id']
    spotify_album[id_track]['name'] = track['name']
    spotify_album[id_track]['popularity'] = track['popularity']
    spotify_album[id_track]['explicit'] = track['explicit']
    spotify_album[id_track]['duration_ms'] = track['duration_ms']
    spotify_album[id_track]['release_date'] = track['album']['release_date']
 
    artists_track = track['artists']
    spotify_album[id_track]['artists'] = []
    for artist in artists_track:
      spotify_album[id_track]['artists'].append(artist['name'])
  return spotify_album


In [185]:
def get_all_albums_songs(albums_ids_names):
  spotify_albums = []
  albums_names = []
  for id, name in albums_ids_names.items():
    if name not in albums_names:
      albums_names.append(name)
      album_songs = get_album_songs(id,name) 
    for item in album_songs.items():
      spotify_albums.append(item[1]) 
  return spotify_albums

In [186]:
def convert_to_csv(filepath, name):
  keys = filepath[0].keys()
  print(keys)
  csv_name = ''+ name + '.csv'
  with open(csv_name, 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(filepath)
  return

In [187]:
name = "NexoAnexo"
artist = get_artist(name)    
if artist:
  artist_id = get_artist_id(artist)
  albums_id_names = get_artist_albums_id_names(artist_id)
  all_albums = get_all_albums_songs(albums_id_names)
  convert_to_csv(all_albums, 'NexoAnexo')   
else:
  logger.error("Can't find artist: %s", artist)

HTTP Error for GET to https://api.spotify.com/v1/tracks/5C1iQCV3xuc0oHQpe7fhhQ returned 404 due to error


SpotifyException: ignored

### **Dicionario dos dados**

Lembrem da definição de popularidade que foi dada lá no volume 01. Uma música nova tem muito mais chance de ser mais popular. 

contexto



**Informações sobre os atributos**


contexto

### **Importar os pacotes e datasets necessário**



contexto

In [ ]:
#importar os pacotes necessarios
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer

# configurar o notebook
sns.set_style('whitegrid');

### **Leitura do arquivo**

Primeiro vamos ler nosso arquivo fora do notebook e tirar algumas informações, de como os dados estão separados e etc..

Abrindo nosso arquivo foi possivel identificar que está separado...Se trata de um arquivo csv

Em alguns projetos reais essa etapa de importar os dados pode ser mais trabalhosa exigindo outros parametros, como header, skiprows e etc..

In [ ]:
dados = 'NexoAnexo.csv'

df = pd.read_csv(dados, sep = ',')

contexto

In [ ]:
print(f'Número de linhas ou variaveis {df.shape[0]}')
print(f'Número de entradas ou colunas {df.shape[1]}')

contexto

## **Limpeza e preparação dos dados**

contexto

### **Deletando colunas**

Como o nosso conjunto de dados contém colunas que não serão uteis para o que desejamos fazer, vamos deletar...

In [ ]:
df.head()   

Vamos deletar as colunas...Isso é importante porque..

In [ ]:
df.columns.unique()

Dessa forma consigo ter uma visão de todas as minhas colunas, então agora é so dar um CTRL + C nas que desejo e inserir no proximo método 

In [ ]:
df = df[['album', 'name', 'track_number', 'popularity', 'duration_ms']].copy()

Costumo deletar as colunas dessa maneira por que me permite escolher uma nova ordem para as colunas, além de não delatarem proprimamente as outras colunas que foram descartadas.

In [ ]:
df.columns

### **Renomeando colunas**

A renomeação das colunas é importante pq...contexto

In [ ]:
df.columns

Para renomear todas as colunas vou utilizar o metodo df.columns...

In [ ]:
df.columns = ['nome_do_album', 'nome_da_faixa',  'posicao_da_faixa', 'popularidade', 'duraçao_da_musica']

Quando o objetivo é renomear uma ou mais colunas, o metodo df.rename seria melhor...

In [ ]:
df.head(3)

contexto

### **Verificar dados ausentes**

contexto

In [ ]:
df.isna().sum()

Deve-se ficar atento pois muita das vezes uma coluna pode ter o registro de dados, ou seja, é informado pelo método que o conjunto de dados não possui valores nulos, entranto, esses dados não passam nenhuma informação util 

Também poderia fazer o uso do isnull, porém recomento utilizar o isna, devido que o final  do método "na", remete ao dropna, fillna e etc... Que são metodos que também trabalham de alguma forma com dados ausentes. 

Vamos usar um "df.head(10) e df.tail(10)", pra analisar com calma os 10 primeiros e ultimos registros, averiguando se os dados realmente passam alguma informação, já que não temos dados ausentes.

In [ ]:
df.head(10)

In [ ]:
df.tail(10)

Contexto...Todos os registros passam alguma informação....

Contexto, Real plug mixtape é o album mais antigo, por isso sua popularidade não tem muitos número, devido que a popularidade está relacionada a musicas ouvidas no momento... Intrigante é a faixa 2020 que ainda se mantém com uma boa popularidade, vamos analisar mais essa faixa na análise exploratoria

De fato 2020 poderia ter sido um ano de sucesso e uma ascensao da faixa 2020, como grande promessa, porem a faixa também fala de anos posteriores, como 2023, quem sabe nesse ano coisas grandes venham a acontecer..

### **Verificar os tipos de colunas** 

contexto... É importante verificar se nenhuma coluna foi preenchida de forma errada, ou seja, a coluna trata-se de uma coluna inteira, mas está como string

In [ ]:
df.dtypes

In [ ]:
df.dtypes.value_counts()

contexto... Temos 2 faixas do tipo objeto e 2 faixas do tipo inteiro

In [ ]:
df.head()

contexto

### **Alterando tipo de colunas**

Nesse projeto nao vamos alterar o tipo das colunas...contexto

Porém se quisessemos alterar, poderiamos fazer da seguinte forma...

In [ ]:
df['popularidade'] = df['popularidade'].astype(float)

In [ ]:
df.dtypes

In [ ]:
df['popularidade'] = df['popularidade'].astype(int)

contexto

### **Corrigindo dados divergentes**

O ponto importante que acho interessante de falar que não aconteceu nesse dataset é sobre dados divergentes, que podem surgir em alguns conjuntos de dados.  Uma forma eficiente de filtrar e remover o que não for necessário é utilizando expressões regulares. Não entraremos em detalhes sobre a utilização dessas funções.

https://medium.com/@lucasoliveiras/limpeza-e-prepara%C3%A7%C3%A3o-dos-dados-com-pandas-856e844abfbb

### **Verificando valores duplicados**

Contexto...Valores duplicados são encontrados com frequência e dependendo da análise que será feita, esses valores podem alterar o resultado final.

https://medium.com/@lucasoliveiras/limpeza-e-prepara%C3%A7%C3%A3o-dos-dados-com-pandas-856e844abfbb

Por se tratar de um dataset com diversas musicas, e numeros, é importante verificar se não tem dados duplicados

In [ ]:
df.duplicated().value_counts()

contexto

### **Outliers**

contexto.. Os outliers são os valores que chamamos de discrepantes.... 

inserir imagens do artigo: https://towardsdatascience.com/a-gentle-introduction-to-exploratory-data-analysis-f11d843b8184

Para identificar a presença de outliers a melhor soluçã é DISTRIBUIÇÃO, que podem ser feitas tanto com gráficos de hsitograma como o proprio gráfico de distribução.

In [ ]:
df.plot(kind='hist')

Artigo que tbm fala sobre outliers = https://r4ds.had.co.nz/exploratory-data-analysis.html

## **Exploração e Visualização de dados**

contexto... Toda exploração de dados está relacionada a hipoteses,  avaliar pressupostos sobre os quais a inferencia estatistica se baseara....

A EDA é um ciclo iterativo. Vocês:

* Gera perguntas sobre seus dados.

* Pesquisa respostas visualizando, transformando e modelando seus dados.

* Usa o que aprendeu para refinar suas perguntas e / ou gerar novas perguntas.

Artigo: https://r4ds.had.co.nz/exploratory-data-analysis.html

Fazer uma exploração sempre temos que ter como objetivo ir atrás de **Perguntas** e **Hipoteses**

Então vamos definir algumas perguntas que queremos responder na analise exploratoria dos dados, para reter informações... A análise exploratoria deve se chegar ao fim, quando for identificado que já se atingiu o objetivo desejado,, na prática isso é definido pela área de negocios.

### **Perguntas:**

* Qual é a música mais popular de Nexo?
* Qual é o álbum mais popular?
* E o single?
* Quantas musicas todos os albuns tem?
* A popularidade das músicas tem relação com a posição?

### **Hipóteses:**

* No album Real Plug, ...
* No album Trap de Cria, acredito que seja OI Fake 
* No album TFF, acredito que seja Match, por mais que a minha preferida seja ...

* Musica mais pouplar, Ratatah 
* Album mais popular, TFF
* **O sucesso Trap de cria**


contexto... Durante a analise exploratoria podem surgir outras perguntas, então é sempre bom começar com perguntas simple e ir aumentando suas complexidades conforme vai conhecendo os dados, o qual também é um dos objetivos da analise exploratoria, você tomar conhecimento dos seus dados.

Vamos começar visualizando novamente nosso dataframe...

In [ ]:
df.head()

### **Q1: Quais músicas dos albumns de NexoAnexo estão popular no momento?** Data = 

Sem contar os singles, aqui estamos levando em consideração as musicas dos albumns...

A popularidade das musicas podem mudar diariamente de acordo com os lançamentos de novas músicas que surgem, as musicas que estão popular no momento sao:

In [ ]:
df[['nome_da_faixa', 'popularidade']].sort_values(ascending=False, by='popularidade').reset_index(drop=True)

**Visualização de dados**

A etapa de visualizacao de dados está dentro da exploração de dados, com o objetivo de transmitir a informação retiradas dos dados... Contexto

contexto sobre visualizacao de dados

Então a primeira coisa que vamos visualizar é a primeira pergunta que foi levantada, quais são as músicas que estão popular no momento?...

In [ ]:
#pegar as 10 músicas que estão popular e transformar em uma variavel para plotar posteriormente
musicas_populares = df[['nome_da_faixa', 'popularidade']].sort_values(ascending=False, by='popularidade').reset_index(drop=True)[:20]

In [ ]:
#plotar gráfico de barras com seaborn das músicas que estão popular
plt.figure(figsize=(16,6))
ax = sns.barplot(x= 'nome_da_faixa', 
                 y = 'popularidade', 
                 data = musicas_populares, 
                 palette = sns.color_palette("BuGn_r", n_colors=len(musicas_populares)))
ax.set_xticklabels(ax.get_xticklabels(), rotation=65)
ax.set_title('As 10 músicas que estão popular no momento');


O que me surpreende nessa primeira visão é a faixa Trap de Cria, que foi um lançamento do album Trap de Cria Mixtape, antes do Trap From Future. Mas não é só isso que dá pra observar, a faixa 2020 do primeiro albúm Real Plug Mix Tape está a frente de muitas da musicas do recente album Trap From Future. 

Um questionamento que tenho é se a popularidade dessas faixas que estão em alta tem relação com a posição das faixas. Tendo em vista, que a primeira faixa sempre é a primeira a tocar obviamente, sempre que inicia a playlist. Mas antes disso vamos separar os albuns por variveis para a analise ficar mais clara e facil.

**Definindo por albumns**

In [ ]:
rmx = df.query("nome_do_album == 'Real Plug Mixtape' ")

In [ ]:
tcm = df.query("nome_do_album == 'Trap de Cria Mixtape' ")

In [ ]:
tff = df.query("nome_do_album == 'Trap from Future' ")

contexto

### **Q2: Quantidade de musicas que estão sendo analisadas?**

In [ ]:
print(f'{df["nome_da_faixa"].count()} músicas') 

**Músicas por album**

Albúm RealPlug MTP:

In [ ]:
print(f'O albúm tem: {rmx["nome_da_faixa"].count()} faixas')

rmx[['posicao_da_faixa', 'nome_da_faixa']].reset_index(drop=True)

contexto

In [ ]:
plt.figure(figsize=(12,6))
sns.lineplot(x='nome_da_faixa', y='popularidade', data=rmx);

Já conseguimos observar aqui que de fato a faixa 2020 é um sucesso, tendo em vista que tem uma popularidade alta mesmo estando na ultima posicao do album. 

O fato de a popularidade das outras faixas estarem baixas são devido que esse album é o mais antigo e o calculo da popularidade é feito a partir....

Albúm Trap de cria:

In [ ]:
tcm[['posicao_da_faixa', 'nome_da_faixa']].reset_index(drop=True)

contexto

In [ ]:
plt.figure(figsize=(12,6))
sns.lineplot(x='nome_da_faixa', y='popularidade', data=tcm);

Trap from future:

In [ ]:
tff[['posicao_da_faixa', 'nome_da_faixa']]

contexto

### **Q3: Verificando se a popularidade das track tem a relação com sua posição**

contexto

**Album: Trap From Future**

In [ ]:
tff[['nome_da_faixa', 'posicao_da_faixa', 'popularidade']]

A primeira faixa é a mais popular, mas sei que a primeira faixa é sempre a primeira a tocar numa playlist do spotify e nem sempre é a que você mais ouve, mas ela torna-se a mais ouvida por sempre que inicia a playlist ela é a primeira a tocar.

Isso também pode ser aplicado aqui, por que se a posição tivesse relação com a popularidade as outras faixas seguintes também deveriam continuar sendo as mais populosas, que não acontece..

contexto, particulamente curto muito a musica Novo Rock com uma pegada diferente e faz lembrar o estilo bate cabeça

In [ ]:
rmx[['nome_da_faixa','posicao_da_faixa', 'popularidade']].reset_index(drop=True)

Como dito anteriormente é possivel observar que a faixa 2020 é de fato um sucesso, por que se analisarmos as musicas que estão na ultima posição de cada album, em sua maioria são as menos ouvidas.

É incrivel como 2020 ainda possui um alcance grande, mesmo que a popularidade esteja pequena, é uma musica que está a frente de grandes faixas do recente album lançado, tff, como a faixa Wow.

In [ ]:
tcm[['nome_da_faixa','posicao_da_faixa', 'popularidade']].reset_index(drop=True)

Assim como no album TFF, a musica mais popular deste album é a que está na primeira posição, vale resaltar que a musica Trap de cria é a musica mais popular de todos os albuns, como foi visto anteriormente.

contexto... O que fica claro é que a popularidade das faixas só tem relação com a posição para a primeira música de cada albúm, devido ao que foi explicado...

Ainda podemos fazer uma correlção da popularidade com a posição das faixas para comprovar se realmente a posição da faixa não tem relação com a popularidade. Aplicando o metodo de correlação de estatistica multivariada.

In [ ]:
df[['posicao_da_faixa', 'popularidade']].corr(method='spearman')

Contexto sobre os metodos de correlação/correlação e estatistica multivariada..

Usamos o metodo spearman de correlação, um dos mais  famosos.... 

Nota-se que existe uma correlação muito fraca da posição da faixa com a popularidade

### **Q5: Qual a classificação da popularidade?**

Agora vou investigar mais essa coluna da popularidade para buscar algumas informações aplicando metodos estatisticos.

Primeiro vamos observar um resumo estatistico dos dados...

In [ ]:
df['popularidade'].describe()

contexto... A popularidade mais alta é 17 e a menor é 0 ....

Agora irei aplicar estatistica descritiva univariada para explorar mais a coluna popularidade...

### **Aplicando estatistica descritiva univariada**

contexto... Sobre estatistica descritiva univariada

### **Q6: Qual a media de popularidade?**

contexto.. Media de todos os albuns

Contexto falar um pouco sobre a media...O calculo da média é feito de forma que... FOTO DA FUNÇÃO

In [ ]:
df['popularidade'].mean()

contexto

### **Q7: Qual a média da popularidade por albúm?**

In [ ]:
media_popularidade = df.groupby('nome_do_album')['popularidade'].mean().reset_index().sort_values(ascending=False, by='popularidade')
media_popularidade 

contexto... Album TFF com popularidade acima da media, isso se da devido que é o album mais recente...

Agora irei aplicar um pouco de engenharia de recursos e criar uma nova coluna de acordo com a popularidade...

### **Construindo uma nova coluna de acordo com a media da popularidade**

É importante construir novas colunas ou novos recursos, por criar outras maneiras de utilizar os dados.... Contexto (Engenharia de Recursos)

In [ ]:
df['popularidade_por_faixa'] = ['Alta' if x > 2.60 else 'Baixo' for x in df['popularidade']]

Se a faixa estiver com uma popularidade acima de 2.60, está com a popularidade alta, tendo em vista que a media da popularidade é 2.60.

Lembrar de alterar isso de acordo com a media atual....

In [ ]:
df.head()

A partir da construção dessa nova coluna podemos trabalhar e usar mais os nossos dados, que é o objetivo da engenharia de recursos, de forma que a partir dessa nova coluna surgem mais perguntas.

Por isso é sempre bom começar com perguntas simples e ir aumentando suas complexidades conforme vai conhecendo e explorandos os dados...

### **Q8: Quantas faixas estão em alta?**

In [ ]:
df['popularidade_por_faixa'].value_counts()

contexto

### **Q9: Quais são as faixas que estão popular no momento?**

In [ ]:
df.query("popularidade_por_faixa == 'Alta'").sort_values(ascending=False, by='popularidade').reset_index()[['nome_do_album', 'nome_da_faixa', 'popularidade', 'popularidade_por_faixa']]

contexto

### **Q10: Quantas faixas que estão em alta não estão na primeira posição do albúm?**

contexto

In [ ]:
df.query("popularidade_por_faixa == 'Alta' & posicao_da_faixa != '1' ").sort_values(ascending=False, by='popularidade').reset_index()[['nome_do_album', 'nome_da_faixa', 'posicao_da_faixa', 'popularidade', 'popularidade_por_faixa']]

Ou seja mais da metade das faixas que estão em alta não estão na primeira posição do albúm... Comprovando o que observamos que a posicao da faixa não tem relação com a popularidade...

### **Aplicando um pouco mais de estatistica descritiva univariada**

contexto... Já aplicamos um pouco de estatistica descritiva univariada quando visualizamos a média da popularidade das faixas, agora vamos aplicar outros metodos estatisticos...

**contruir um grafico para cada metodo estatistico de acordo com os estudos**

contexto

**Mediana**

contexto sobre mediana

In [ ]:
df.median()

**Moda**

contexto sobre mode

In [ ]:
df.mode()

**Desvio padrão**

contexto sobre desvio padrao

In [ ]:
#desvio padrao
df.std()

**Box plot**

contexto sobre box plot....

In [ ]:
#box plot

sns.boxplot(y='nome_da_faixa', x='popularidade', data=tff)

**Assimetria**

contexto sobre assimetria...

In [ ]:
#Assimetria

df.skew()

**Kurtossis**

contexto sobre kurtose...

In [ ]:
#Kurtose
df.kurt()

A exploração dos dados deve chegar ao fim quando o pessoal da área de negocios se sentirem satisfeitos ou quando os objetivos das informações retiradas dos dados forem alcançados.....

### **Perguntas e hipoteses respondidas na analise exploratoria de dados:**

* Q1 = 
* Q2 = 
* Q3 = 

## **Visualizando os dados**

# Ver se coloco a visualizacao de dados dentro da exploracao ; ver como é encaixada em outros projetos do carlos melo

INVESTIR TEMPO NA VISUALIZAÇÃO DE DADOS, TANTO PRA APRENDER COMO PRA DEIXAR BONITO

contexto

In [ ]:
import seaborn as sns

In [ ]:
sns.set_style('whitegrid')

### **Grafico musica mais popular**

In [ ]:
musica_mais_popular = df[['nome_da_faixa', 'popularidade']].sort_values(ascending=False, by='popularidade').reset_index(drop=True)[:10]

In [ ]:
plt.figure(figsize=(15,6))
ax = sns.barplot(x='nome_da_faixa', y ='popularidade', data=musica_mais_popular)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
ax.set_title('10 musicas mais populares dos albumns')

### **Distribuição da popularidade das musicas de acordo com suas posições**

In [ ]:
dados = df.query('popularidade >= 2').sort_values(ascending=False, by='popularidade').reset_index()[['nome_do_album','posicao_da_faixa', 'nome_da_faixa', 'popularidade', 'popularidade_por_faixa']]

Musicas com popularidade acima de 2, tendo em vista que a media da popularidade é 2

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(x='posicao_da_faixa', y = 'popularidade', hue= 'nome_da_faixa', data=dados)

## **Deploy com Python, streamlit +heroku**

### **Aplicação com streamlit**

Explicar streamlit, framework do python excelente para construir dashboard de visualização de dados....

### **Colocar disponivel com Heroku**

Ensinar os passos a passos para colocar a aplicação no ar com heroku: 

https://medium.com/@gustavo_domingues/como-fazer-deploy-de-sua-aplica%C3%A7%C3%A3o-streamlit-no-heroku-5fc1d2609df7#:~:text=email%40dominio.com-,Deploy,site%2C%20ent%C3%A3o%20escolha%20algo%20legal.&text=Sempre%20que%20quiser%20atualizar%20ou,basta%20seguir%20os%20passos%20abaixo.

https://www.youtube.com/watch?v=mQ7rGcE766k&list=WL&index=2&t=0s




contexto

## **Como ajuda o negócio?**